In [1]:
import logging, os
logging.disable(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

from pathlib import Path

import keras_tuner as kt

from importlib import reload
import hyper_model_creator

In [2]:
trials_dir = Path('trials', 'hyperband_median_batch')
trial_metric = 'val_weightedf2score'

In [ ]:
reload(hyper_model_creator)

hypermodel = hyper_model_creator.BuildHyperModel(trials_dir, trial_metric)

tuner = kt.Hyperband(
    hypermodel,
    objective=kt.Objective(trial_metric, direction='max'),
    directory=trials_dir.parent,
    project_name=trials_dir.name,
    hyperband_iterations=3,
    max_epochs=20,
    overwrite=False,
)
     
tuner.search(verbose=1)

Trial 25 Complete [00h 24m 31s]
val_weightedf2score: 0.6208455562591553

Best val_weightedf2score So Far: 0.6277794241905212
Total elapsed time: 08h 32m 44s

Search: Running Trial #26

Value             |Best Value So Far |Hyperparameter
8                 |8                 |batch_size
True              |True              |class_weight
2017              |2017_2018_2019    |training_years
average           |max               |pooling
5                 |5                 |filters_power
3                 |3                 |kernel_size
4                 |3                 |units_power
0.2               |0.2               |dropout_rate
True              |False             |output_bias
binary_crossent...|binary_focal_cr...|loss
20                |20                |tuner/epochs
7                 |7                 |tuner/initial_epoch
1                 |2                 |tuner/bracket
1                 |2                 |tuner/round
0018              |0012              |tuner/trial_id

Ep

/home/pj/miniconda3/envs/tf/lib/python3.10/site-packages/keras/src/saving/saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 46 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


   18/28401 ━━━━━━━━━━━━━━━━━━━━ 2:59 6ms/step - accuracy: 0.4969 - auc: 0.8666 - loss: 0.3653 - prc: 0.6563 - precision: 0.6987 - recall: 0.5235 - weightedf1score: 0.5561 - weightedf2score: 0.5331  

I0000 00:00:1722977397.211133     381 asm_compiler.cc:369] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_2', 1620 bytes spill stores, 1524 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion_1', 40 bytes spill stores, 40 bytes spill loads



28401/28401 ━━━━━━━━━━━━━━━━━━━━ 168s 6ms/step - accuracy: 0.4595 - auc: 0.8809 - loss: 0.3562 - prc: 0.7186 - precision: 0.7264 - recall: 0.5506 - weightedf1score: 0.5851 - weightedf2score: 0.5628 - val_accuracy: 0.4568 - val_auc: 0.8880 - val_loss: 0.3471 - val_prc: 0.7399 - val_precision: 0.7306 - val_recall: 0.5733 - val_weightedf1score: 0.5912 - val_weightedf2score: 0.5783
Epoch 9/20
20334/28401 ━━━━━━━━━━━━━━━━━━━━ 42s 5ms/step - accuracy: 0.4601 - auc: 0.8830 - loss: 0.3541 - prc: 0.7239 - precision: 0.7288 - recall: 0.5560 - weightedf1score: 0.5901 - weightedf2score: 0.5680

In [ ]:
# tuner.get_best_models(num_models=5)[0].fit(
#     x=training_generator,
#     validation_data=testing_generator,
#     class_weight=data_summary['class_weights'],
#     epochs=10,
#     verbose=1
# )

In [ ]:
tuner.results_summary(num_trials=10)